<a href="https://colab.research.google.com/github/RafsanJany-44/Research-Wake-NREM-REM/blob/main/scratchpad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install imbalanced-learn

from imblearn.over_sampling import SMOTE

def balance(X_temp, y_temp):
  smote = SMOTE()
  X_temp, y_temp= smote.fit_resample(X_temp, y_temp)

  return pd.concat([pd.DataFrame(X_temp), pd.DataFrame(y_temp)], axis=1)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np

dataset = pd.read_csv('https://raw.githubusercontent.com/RafsanJany-44/Thesis_Project/main/All_DATA/without_sn_and_Epoch/EEG_HMC.csv')
dataset.head()
target="Sleep Stage"

In [4]:
dataset[target] = dataset[target].replace(['W'],'Wake')
dataset[target] = dataset[target].replace(['N1'],'NREM')
dataset[target] = dataset[target].replace(['N2'],'NREM')
dataset[target] = dataset[target].replace(['N3'],'NREM')
dataset[target] = dataset[target].replace(['R'],'REM')

In [5]:
dataset[target].value_counts()

classes = np.array(sorted(list(set(dataset[target]))))

In [6]:
classes

array(['NREM', 'REM', 'Wake'], dtype='<U4')

In [7]:
X = dataset.loc[:, dataset.columns!=target]
y = dataset[target]

In [8]:
#Splitting dataset into training and testing dataset
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=0)

In [9]:
y_train.value_counts()

NREM    48746
Wake    12964
REM     10952
Name: Sleep Stage, dtype: int64

In [10]:
new_dataset=balance(X_train,y_train)

X_train = new_dataset.loc[:, new_dataset.columns!=target]
y_train = new_dataset[target] 

In [11]:
y_train.value_counts()

NREM    48746
Wake    48746
REM     48746
Name: Sleep Stage, dtype: int64

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
from tqdm import tqdm
N=2000
k_range = range (900,N+1,50)
scores={}
scores_list = []
for k in tqdm(k_range):
  xgb_classifier = xgb.XGBClassifier(n_estimators=k,random_state=0)
  xgb_classifier.fit(X_train, y_train)
  y_pred=xgb_classifier.predict(X_test)
  scores[k] = accuracy_score(y_test,y_pred)
  scores_list.append(accuracy_score(y_test,y_pred))
  print(str(k)+"/"+str(N)+" round completed......................... Accurecy: "+str(accuracy_score(y_test,y_pred)))


%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize = (25,10))
plt.plot(k_range,scores_list)
plt.xlabel('Value of n_estimators')
plt.ylabel ('Testing Accuracy')



print("The best n_estimators:")
best_estimator=list(scores.keys())[scores_list.index(max(scores_list))]
print(best_estimator)

  4%|▍         | 1/23 [25:55<9:30:29, 1555.89s/it]

900/2000 round completed......................... Accurecy: 0.8795160524183409


  9%|▊         | 2/23 [53:16<9:21:59, 1605.68s/it]

950/2000 round completed......................... Accurecy: 0.8803263572606108


 13%|█▎        | 3/23 [1:22:11<9:14:56, 1664.83s/it]

...

1150/2000 round completed......................... Accurecy: 0.8833440442594093


 30%|███       | 7/23 [3:31:34<8:33:26, 1925.42s/it]

1200/2000 round completed......................... Accurecy: 0.8841822906479645


 35%|███▍      | 8/23 [4:07:50<8:21:16, 2005.13s/it]

1250/2000 round completed......................... Accurecy: 0.8843499399256755


In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
N=30
k_range = range (1,N+1)
scores={}
scores_list = []
for k in tqdm(k_range):
  xgb_classifier = xgb.XGBClassifier(max_depth=k,random_state=0)
  xgb_classifier.fit(X_train, y_train)
  y_pred=xgb_classifier.predict(X_test)
  scores[k] = accuracy_score(y_test,y_pred)
  scores_list.append(accuracy_score(y_test,y_pred))
  print(str(k)+"/"+str(N)+" round completed......................... Accurecy: "+str(accuracy_score(y_test,y_pred)))


%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize = (25,10))
plt.plot(k_range,scores_list)
plt.xlabel('Value of n_estimators')
plt.ylabel ('Testing Accuracy')



print("The best depth:")
best_depth=list(scores.keys())[scores_list.index(max(scores_list))]
print(best_depth)